<a href="https://colab.research.google.com/github/claudia-hm/IRMAS_DeepLearning/blob/master/IRMAS_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project : Instrument recognition in musical audio signals

# 0. Preliminaries

### Mount drive

In [1]:
##Opening Data And saving

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls '/content/drive/My Drive/DeepLearning_2020/Final Project/Data/'

data_path = '/content/drive/My Drive/DeepLearning_2020/Final Project/Data/'
results_path = '/content/drive/My Drive/DeepLearning_2020/Final Project/Results/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cel			  flu		 labels.npy  sax	   vio
cla			  gac		 org	     specs_11.npy  voi
data_11_instruments.json  gel		 pia	     specs.npy
data_jason.json		  labels_11.npy  README.txt  tru


### Install required packages

In [0]:
%%capture
!pip install essentia

### Imports

In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image

%tensorflow_version 2.x
import tensorflow as tf
import timeit

import os
import json

import sys
from tqdm.auto import tqdm
import essentia.standard as es
from essentia import Pool
import sklearn

import random
import librosa.display
from IPython.display import Audio

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


Using TensorFlow backend.


### Warm up CPU and GPU

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):


# 1. Data loading and pre-processing

###Store filenames in dataset

This dictionary maps folder names (alias for musical instruments) to numerical indexes.

In [0]:
CLASS_MAPPING = {
    'flu': 0, 
    'tru': 1, 
    'voi': 2, 
    'gac': 3, 
    'pia': 4
}

First, we store the filenames in a dictionary, where the keys are the folder names (as in CLASS_MAPING)

In [0]:
audio_files = dict()

for i in CLASS_MAPPING.keys():
  files = []
  for filename in os.listdir(data_path + "/" + i + "/"):      
    files.append(filename)
  audio_files[i] = files
print(audio_files)

###Save json file in the Data folder

Then, we save the previously created dictionary in a json file

In [0]:
with open(data_path +'/data_jason.json', 'w') as outfile:
    json.dump(audio_files, outfile)

### Functions to compute spectogram

To classify samples we will compute their Log Mel Spectrograms using Essentia library. These are Spectrogram with the Log Mel Scale as its y axis.  This Mel Scale is constructed such that sounds of equal distance from each other on the Mel Scale, also “sound” to humans as they are equal in distance from one another. (https://towardsdatascience.com/getting-to-know-the-mel-spectrogram-31bca3e2d9d0)

In [0]:
def pad(l, sr):
    # 0-Pad 10 sec at fs hz and add little noise
    z = np.zeros(10*sr, dtype='float32')
    z[:l.size] = l
    z = z + 5*1e-4*np.random.rand(z.size).astype('float32')
    return z


def compute_spectrogram(filename, sr=16000, n_mels=64):
    # compute log mel magnitude spectrogram 
    audio = es.MonoLoader(filename=filename,sampleRate=sr)()
    audio = pad(audio, sr)
    
    # essentia extractor
    windowing = es.Windowing(type='hann', normalized=False, zeroPadding=0)
    spectrum = es.Spectrum()
    melbands = es.MelBands(numberBands=n_mels, 
                           sampleRate=sr, 
                           lowFrequencyBound=0, 
                           highFrequencyBound=8000, 
                           inputSize=(2048)//2+1, 
                           weighting='linear', 
                           normalize='unit_tri', 
                           warpingFormula='slaneyMel', 
                           type='power')
    
    norm10k = es.UnaryOperator(type='identity', shift=0, scale=1)
    log10 = es.UnaryOperator(type='log10')
    results = Pool()

    for frame in es.FrameGenerator(audio, frameSize=1024, hopSize=512, startFromZero=False):
        spectrumFrame = spectrum(windowing(frame))     
        results.add('melbands', log10(norm10k(melbands(spectrumFrame))))
    
    mel_log_spectrum = results['melbands']
    return mel_log_spectrum[:64].T

### Example of computing spectrogram

Here, we show an example of computing a spectrogram from a random file in the dataset. Also, the random track can be reproduced.

In [0]:
#select random file
random_instrument = random.choice(list(CLASS_MAPPING.keys()))
rand_sound = random.choice(audio_files[random_instrument])
data_rand_instrument = data_path + '/' + random_instrument + '/'


#compute spectrogram and reproduce the track
x = compute_spectrogram(f'{data_rand_instrument}/{rand_sound}')
librosa.display.specshow(x, sr=16000, x_axis='time', y_axis='mel')

print(rand_sound)
Audio(f'{data_rand_instrument}/{rand_sound}')

### Compute spectrogram for whole dataset

If this cell is activated to true, it will recompute all the spectrograms for the files in the dataset. It takes approximately 30 minutes to complete. For this reason, we have stored them in the file specs.npy and the labels in labels.npy.

In [0]:
if False:
    dataset = json.load(open(data_path+'/data_jason.json', 'rb'))
    num_sounds = sum([len(v) for _, v in dataset.items()])
    num_classes = len(dataset)

    specs = np.zeros((num_sounds, 64, 64))
    labels = np.zeros((num_sounds,))

    idx = 0

    with tqdm(total=num_sounds) as pbar:
        for class_name, sounds in dataset.items():
            for sound in sounds:
                specs[idx] = compute_spectrogram(f'{data_path}{class_name}/{sound}')
                # labels[idx][NSYNTH_CLASS_MAPPING[class_name]] = 1
                labels[idx] = CLASS_MAPPING[class_name]
                idx += 1
                pbar.update()

    np.save(data_path+'specs.npy', specs)
    np.save(data_path+'labels.npy', labels)

# 2. Create custom datasaet

First, we load the numpy files that contain the spectrograms and labels.

In [0]:
specs = np.load(data_path+'specs.npy')
labels = np.load(data_path+'labels.npy')

#reshape the spectrograms
specs = specs.reshape(-1, 64, 64, 1)
#specs = (specs - np.mean(specs)) / np.std(specs)

# split train test
X_train, X_test, y_train, y_test = train_test_split(specs, labels, test_size=0.2, random_state=42)

print("There are " + str(len(X_train)) + " training samples and " + str(len(X_test))+ " testing samples.")


Then, following the procedure presented in S2, we have created a Dataset class for the IRMAS data.

In [0]:
#Making native class loader
class IRMAS(torch.utils.data.Dataset):
    # Initialization method for the dataset
    def __init__(self, X_train, y_train):
        self.data = torch.from_numpy(X_train.astype(float))
        self.labels = torch.from_numpy(y_train.astype(float)).squeeze()

    
    # What to do to load a single item in the dataset (read image and label)    
    def __getitem__(self, index):
        data = self.data[index]
        lbl = self.labels[index]

        data = np.transpose(data, axes=[2,0,1])

        return data,lbl
    
    
        pass
    
    # Return the number of images
    def __len__(self):
        return len(self.data)
      

We instantiate the dataset for training and the train_loader.

In [0]:
IRMASTrain = IRMAS(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=IRMASTrain,
                                               batch_size=32,  
                                               shuffle=True)


Here we show an example of usage of the data loader, extracting spectrograms from our dataset.

In [0]:
for images, labels in train_loader:
  print(images.shape, labels.shape) #images, labels shape
  for img,lbl in zip(images,labels): 
    print('lbl : ',lbl)
    librosa.display.specshow(img.squeeze().numpy(), sr=16000, x_axis='time', y_axis='mel')
    plt.show()
  break

# 3. Define the model

Our model is based in the VGG network. The main characteristics are: 3x3 convolutional filters, fully connected layers at the end, and ReLU activation functions.

In [0]:
class VGG(nn.Module):
    def __init__(self, num_classes=5):
        
        super(VGG, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.batch1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.batch2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.batch3 = nn.BatchNorm2d(64)


        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.batch4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.batch5 = nn.BatchNorm2d(128)
        
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.batch6 = nn.BatchNorm1d(2*2*128)
        self.fc1 = nn.Linear(2*2*128, 64)
        self.fc2 = nn.Linear(64, num_classes)
        
        self.maxpool= nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim = 1)

        self.dropout2d = nn.Dropout2d(0.05)
        
    def forward(self, x):

        out = self.relu(self.conv1(x))
        out = self.batch1(out)
        out = self.maxpool(out) # 32x32x16 --> 1x16x32x32

        out = self.relu(self.conv2(out))
        out = self.batch2(out)
        out = self.maxpool(out) # 16x16x32

        out = self.relu(self.conv3(out))
        out = self.batch3(out)
        out = self.maxpool(out) # 8x8x64

        out = self.relu(self.conv4(out))
        out = self.batch4(out)
        out = self.maxpool(out) # 4x4x128

        out = self.relu(self.conv5(out))
        out = self.batch5(out)
        out = self.maxpool(out) # 2x2x128

        out = out.view(out.size(0), -1) # 512

        out = self.dropout(out) # 512

        out = self.batch6(out) # 512
        
        out = self.relu(self.fc1(out)) # 64

        out = self.softmax(self.fc2(out)) # num_classes

        return out

Here we instantiate our model and print the information related to each of the layers.

In [0]:
vgg = VGG()
vgg = vgg.cuda()
print(vgg)

# 4. Creating loss function, optimizers and hyperparameters

For this task we use Cross Entropy Loss and Adam optimizer. The learning rate is set to 0.001 and the number of epochs to 30.

In [0]:
# Cross entropy loss for classification problems
criterion = nn.CrossEntropyLoss()

#Initialize optimizer 
learning_rate = .001
optimizer = torch.optim.Adam(vgg.parameters(),lr = learning_rate)

# Device configuration (choose GPU if it is available )
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 40

# 5. Actual training

In this section, we train our model using the described hyperparameters and the IRMAS dataset.

In [0]:
# Train the model
if True :
    IRMASTrain = IRMAS(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(dataset=IRMASTrain,
                                               batch_size=64,    
                                               shuffle=True)
    vgg.train() # Set the model in train mode
    total_step = len(train_loader)
    
    loss_list = []

    # Iterate over epochs
    for epoch in range(num_epochs):
        total_loss = 0
        # Iterate the dataset
        for i, (images, labels) in enumerate(train_loader):
            # Get batch of samples and labels
            images = images.to(device).float()
            labels = labels.to(device).float()

            # Forward pass
            outputs = vgg(images)
            loss = criterion(outputs, labels.long())
            total_loss+=loss
                        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % 20 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
        avg_loss = total_loss/total_step
        loss_list.append(avg_loss)
    
    # Save the model checkpoint
    torch.save(vgg.state_dict(), results_path+'/vgg_model.ckpt')
    #to load : model.load_state_dict(torch.load(save_name_ori))


In [0]:
plt.figure(figsize=(10, 8))
plt.plot(loss_list)
plt.title("Loss vs iterations")
plt.ylabel("Loss")
plt.xlabel("Iteration")
plt.show() 

# 6. Analysing the output incluiding testing

Test the model with the test data and compute accuracy.

In [0]:
#Load the model
vgg.load_state_dict(torch.load(results_path+'/vgg_model.ckpt'))

# Test the model
if True : 
    # Load test dataset
    IRMASTest = IRMAS(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(dataset=IRMASTest,
                                               batch_size=16, 
                                               shuffle=True)
    vgg.eval() # Set the model in evaluation mode
    
    # Compute testing accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device).float()
            labels = labels.to(device).float()
            # get network predictions
            outputs = vgg(images)

            # get predicted class
            _, predicted = torch.max(outputs.data, 1)

            # compare with the ground-truth
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        print('Test Accuracy of the model on the 633 test images: {} %'.format(100 * correct / total))